### Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language
lang_pair = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language 

In [4]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [5]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Dutch/Dutch_And_Turkish (tur)_Etymologeek_Result_All.xlsx']

In [6]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,dict_entry_main,turkish_word
0,politie,polis
1,agent,ajan
2,school,okul
3,scholen,okul
4,dokter,doktor
...,...,...
157,chromosoom,kromozom
158,pasja,paşa
159,these,tez
160,bladmuziek,nota


In [7]:
disable_list = []

In [8]:
words = df_pair["dict_entry_main"].values.tolist()

In [9]:
disable_set = set(disable_list)
words_set = set(words)

In [10]:
word_list = list(words_set.difference(disable_set))

In [11]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)
    

#### Concat Result

In [12]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,magazijn
1,checken
2,whisky
3,actrice
4,controle
...,...
157,tennis
158,aubergine
159,kaarten
160,douche


In [13]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,politie agent,320
1,koffie thee,70
2,chocolade cake,53
3,thee koffie,42
4,massage salon,38
...,...,...
135,dokter coach,3
136,astronomie theater,3
137,gitaar techniek,3
138,televisie video,3


In [14]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,thee koffie whisky,4
1,thee koffie cognac,3
2,koffie cognac cake,3
3,karamel chocolade cake,3


In [15]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,thee koffie cognac cake,3


In [16]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram


In [17]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,politie agent,46
1,koffie thee,23
2,thee koffie,22
3,chocolade cake,21
4,koffie cake,13
5,massage salon,10
6,controle magazijn,10
7,jury score,9
8,koffie suiker,8
9,chic stijlvol,7


In [18]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,magazijn,politie agent,320.0,thee koffie whisky,4.0,thee koffie cognac cake,3.0,NaN,NaN,politie agent,46.0
1,checken,koffie thee,70.0,thee koffie cognac,3.0,NaN,NaN,NaN,NaN,koffie thee,23.0
2,whisky,chocolade cake,53.0,koffie cognac cake,3.0,NaN,NaN,NaN,NaN,thee koffie,22.0
3,actrice,thee koffie,42.0,karamel chocolade cake,3.0,NaN,NaN,NaN,NaN,chocolade cake,21.0
4,controle,massage salon,38.0,NaN,NaN,NaN,NaN,NaN,NaN,koffie cake,13.0
...,...,...,...,...,...,...,...,...,...,...,...
157,tennis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,aubergine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,kaarten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,douche,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx", index=False)

In [20]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [21]:
df_word_order_twogram = word_in_twogram(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_twogram(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_twogram(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_twogram(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_twogram(df_shared_all, "word", "sentence") 

In [22]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [23]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [24]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,actrice,actrice kwaliteit,NaN,NaN,NaN,NaN
1,agent,"politie agent, douche agent, agent dossier, ag...",NaN,NaN,NaN,politie agent
2,alcohol,"alcohol controle, liter alcohol, stress alcoho...",NaN,NaN,NaN,NaN
3,ananas,"ananas sex, ananas chocolade",NaN,NaN,NaN,NaN
4,appartement,"luxe appartement, chic appartement",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
91,voetbal,"voetbal coach, voetbal kampioen",NaN,NaN,NaN,voetbal coach
92,whiskey,whiskey koffie,NaN,NaN,NaN,NaN
93,whisky,"koffie whisky, whisky seks, muziek whisky, whi...",thee koffie whisky,NaN,NaN,"koffie whisky, thee koffie whisky, whisky cognac"
94,yoghurt,"yoghurt koffie, chocolade yoghurt",NaN,NaN,NaN,NaN


In [25]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency.xlsx", index=False)